# Transformation

> Often we want to transform inputs as they are passed from one component to another.<br>
> 通常我们想要转变inputs在他从一个组件送到另一个组件的时候

> As an example, we will create a dummy transformation that takes in a super long text, filters the text to only the first 3 paragraphs, and then passes that into a chain to summarize those.

> 例如，我们将创建一个虚拟转换，它接收超长文本，将文本过滤为仅前 3 段，然后将其传递到链中以总结这些内容。

In [1]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Summarize this text:

{output_text}

Summary:"""
)

In [3]:
with open('./input/state_of_the_union.txt') as f:
    state_of_the_union = f.read()

## LCEL

> With LCEL this is trivial, since we can add functions in any RunnableSequence.<br>
>  用LCEL实现很简单, 因为可以为任何RunnableSequence添加函数

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

In [5]:
runnable = (
    {"output_text": lambda text: "\n\n".join(text.split("\n\n")[:3])}
    | prompt
    | ChatOpenAI()
    | StrOutputParser()
)
runnable.invoke(state_of_the_union)

'The speaker addresses the audience, acknowledging the presence of important figures and emphasizing unity among Americans. They mention the impact of COVID-19 and express gratitude for being able to gather again. The speaker emphasizes that, despite political differences, they are united as Americans.'

## Legacy] TransformationChain

In [6]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.llms import OpenAI

> LCEL的实现显然更简单,仅需要一个字典即可

In [7]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    shortened_text = "\n\n".join(text.split("\n\n")[:3])
    return {"output_text": shortened_text}


transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=transform_func
)

In [8]:
template = """Summarize this text:

{output_text}

Summary:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

In [9]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

In [10]:
sequential_chain.run(state_of_the_union)

' In a speech to the nation, the speaker acknowledges the difficulties of the previous year due to COVID-19 and expresses hope for being together again. They emphasize the importance of transcending political divides and uniting as Americans.'